In [1]:
#repeat from parts 1 and 2
!pip install lxml

import pandas as pd
urlx="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

df=pd.read_html(urlx)[0]

#apply Borough requirement -
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df2 = df[~df['Borough'].str.contains('Not assigned', na=False)]

#More than one neighborhood can exist in one postal code area. 
#For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df2= df2.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

#apply Neighbourhood requirement
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
#So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
df2.Neighbourhood = df2.Neighbourhood.fillna(df2['Borough'])
df2.head(10)  

#apply Borough requirement
df2 = df[~df['Borough'].str.contains('Not assigned', na=False)]

url = "http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
dfcord = pd.read_csv(url)
dfcord.head()

df2 = df2.rename(columns={'Postcode': 'Postalcode'})
dfcord = dfcord.rename(columns={'Postal Code': 'Postalcode'})                       


#Merge tables to get desire format

df3 = df2.merge(dfcord, on='Postalcode', how='left')
df3.head(10)



,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Not assigned,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


In [4]:
# first setup some requirements
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [5]:
CLIENT_ID = 'MOX0CWGVCFBOWG2OSIFDRDCPELQFS34UG1OJGCZ2MZL5A0I5' # your Foursquare ID
CLIENT_SECRET = 'DAVRLB4BO1N1CY40NGKDG3B4YSCZLRG125FXP0ABKOUDJF0O' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MOX0CWGVCFBOWG2OSIFDRDCPELQFS34UG1OJGCZ2MZL5A0I5
CLIENT_SECRET:DAVRLB4BO1N1CY40NGKDG3B4YSCZLRG125FXP0ABKOUDJF0O


In [6]:
# Search for data containing "Toronto"
Toronto_data = df3[df3['Borough'].str.contains('Toronto', na=False)].reset_index(drop=True)
Toronto_data.head()


,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [18]:
# create a map of downtown Toronto
latitude = Toronto_data['Latitude'].iloc[0]
longitude= Toronto_data['Longitude'].iloc[0]
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=13)


In [19]:
# add markers to the map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
# and show the map
map_Toronto


In [21]:
# set starting location for foursquare
long1 = Toronto_data.loc[0, 'Longitude'] 
lat1 = Toronto_data.loc[0, 'Latitude'] 


In [25]:
# get some venues in the area
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v=20180604'.format(CLIENT_ID, CLIENT_SECRET, lat1, long1)

results = requests.get(url).json()


In [28]:
#print(results)

In [27]:
#get relevant data from results
# assign relevant part of JSON to venues
venues = results['response']['venues']
# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()


,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4bc70f5d14d7952126a066e9,420 king st E,CA,Toronto,Canada,NaN,75,"[420 king st E, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65465604258614...",43.654656,-79.359871,NaN,NaN,ON,Sackville Playground,v-1573157285,NaN
1,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,4d067a2b26adb1f702f9dc70,475 King St. E,CA,Toronto,Canada,NaN,17,"[475 King St. E, Toronto ON M5A 1L6, Canada]","[{'label': 'display', 'lat': 43.65424167246142...",43.654242,-79.360853,NaN,M5A 1L6,ON,Thruway Muffler / Downtown Brake,v-1573157285,NaN
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,53b8466a498e83df908c3f21,368 King St E,CA,Toronto,Canada,at Trinity St,122,"[368 King St E (at Trinity St), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65355870959944...",43.653559,-79.361809,NaN,NaN,ON,Tandem Coffee,v-1573157285,NaN
3,"[{'id': '4bf58dd8d48988d124951735', 'name': 'A...",False,4dc9d4d9d16495ca5add0803,475 King Street East,CA,Toronto,Canada,Sackville Street,10,"[475 King Street East (Sackville Street), Toro...","[{'label': 'display', 'lat': 43.65419500779484...",43.654195,-79.360545,NaN,M4M 2T7,ON,Cam's Auto Service,v-1573157285,NaN
4,"[{'id': '4f2a23984b9023bd5841ed2c', 'name': 'M...",False,55e8cc7a498e795a53d81d36,NaN,CA,Toronto,Canada,NaN,1684,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.6484050020478,...",43.648405,-79.379919,NaN,NaN,ON,TTC Streetcar #503 Kingston Rd,v-1573157285,NaN


In [30]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered[0:5]

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Sackville Playground,Park,420 king st E,CA,Toronto,Canada,NaN,75,"[420 king st E, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65465604258614...",43.654656,-79.359871,NaN,NaN,ON,4bc70f5d14d7952126a066e9
1,Thruway Muffler / Downtown Brake,Building,475 King St. E,CA,Toronto,Canada,NaN,17,"[475 King St. E, Toronto ON M5A 1L6, Canada]","[{'label': 'display', 'lat': 43.65424167246142...",43.654242,-79.360853,NaN,M5A 1L6,ON,4d067a2b26adb1f702f9dc70
2,Tandem Coffee,Coffee Shop,368 King St E,CA,Toronto,Canada,at Trinity St,122,"[368 King St E (at Trinity St), Toronto ON, Ca...","[{'label': 'display', 'lat': 43.65355870959944...",43.653559,-79.361809,NaN,NaN,ON,53b8466a498e83df908c3f21
3,Cam's Auto Service,Automotive Shop,475 King Street East,CA,Toronto,Canada,Sackville Street,10,"[475 King Street East (Sackville Street), Toro...","[{'label': 'display', 'lat': 43.65419500779484...",43.654195,-79.360545,NaN,M4M 2T7,ON,4dc9d4d9d16495ca5add0803
4,TTC Streetcar #503 Kingston Rd,Moving Target,NaN,CA,Toronto,Canada,NaN,1684,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.6484050020478,...",43.648405,-79.379919,NaN,NaN,ON,55e8cc7a498e795a53d81d36


In [32]:
# add red circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(map_Toronto)


In [33]:
# and show the map
map_Toronto